In [ ]:
https://www.trivago.fr/fr/srl/h%C3%B4tels-barcelone-espagne?search=200-13437;dr-20231102-20231119

 # trivago.com
 

In [1]:


# Scrappe and Clean
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import urllib
import re
import requests
import csv
import time

from selenium.webdriver.common.service import Service
from selenium.webdriver.common.service import Service
from fake_useragent import UserAgent
from urllib3.exceptions import NewConnectionError
import logging
from datetime import datetime, timedelta


import pandas as pd 
import numpy as np 


from selenium.webdriver.common.action_chains import ActionChains


In [ ]:
ua = UserAgent()
user_agent = ua.random

pays = ["Allemagne", "France", "Italie", "Espagne", "Royaume-Uni", "Portugal", "Pays-Bas", "Belgique", "Irlande", "Danemark", "Norvège", "Suède", "Finlande", "Islande", "Russie", "Pologne", "République tchèque", "Hongrie", "Autriche", "Grèce", "Roumanie", "Bulgarie", "Serbie", "Croatie", "Albanie", "Macédoine du Nord", "Monténégro", "Kosovo", "Estonie", "Lettonie", "Lituanie", "Ukraine", "Biélorussie", "Moldavie", "Géorgie", "Arménie", "Azerbaïdjan", "Turquie", "Chypre", "Malte", "Saint-Marin"]
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
options.add_argument(f'user-agent={user_agent}')

all_data = []

logging.basicConfig(filename='scraping.log', level=logging.ERROR)

# Date initiale
start_date = datetime(2023, 11, 22)
end_date = datetime(2023, 11, 28)

# Date finale (2 mois plus tard)
end_date_final = datetime(2024, 1, 22)

# Boucle sur les semaines
while start_date <= end_date_final:
    for i in range(0, len(villes), 2):
        ville1 = villes[i]
        ville2 = villes[i + 1]

        print(f'Début scrape des villes {ville1} et {ville2} pour la semaine du {start_date.strftime("%Y-%m-%d")} au {end_date.strftime("%Y-%m-%d")}')

        
        driver = webdriver.Firefox(options=options)
        try:
            driver.get(f'https://www.momondo.fr/flight-search/{ville1}-{ville2}/{start_date.strftime("%Y-%m-%d")}/{end_date.strftime("%Y-%m-%d")}?sort=price_a')
        except NewConnectionError:
            logging.error(f'Could not establish connection for cities {ville1} and {ville2}, retrying...')
            time.sleep(10)
            try:
                driver.get(f'https://www.momondo.fr/flight-search/{ville1}-{ville2}/{start_date.strftime("%Y-%m-%d")}/{end_date.strftime("%Y-%m-%d")}?sort=price_a')
            except Exception as e:
                logging.error(f'Failed to scrape cities {ville1} and {ville2}: {e}')
                driver.quit()
                continue
        time.sleep(2)

        # Fonction auxiliaire pour récupérer des données
        def extract_data(element, xpath):
            try:
                data = element.find_element(By.XPATH, xpath).get_attribute('textContent')
            except:
                data = ""
            return data
        time.sleep(2)
        # Récupérer le contenu des divs JWEO et xdW8 dans c3J0r-container
        # Récupérer le contenu des éléments
        voyages = driver.find_elements(By.XPATH, './/div[contains(@class, "nrc6-inner")]')


        for voyage in voyages:
        # Récupérer le contenu des éléments pour chaque voyage
            horaires = extract_data(voyage, './/div[@class="vmXl vmXl-mod-variant-large"]')
            trajet = extract_data(voyage, './/div[@class="c_cgF c_cgF-mod-variant-full-airport"]')
            type_trajet = extract_data(voyage, './/span[@class="JWEO-stops-text"]')
            temps_trajet = extract_data(voyage, './/div[@class="xdW8 xdW8-mod-full-airport"]/div[@class="vmXl vmXl-mod-variant-default"]')
            prix = extract_data(voyage, './/div[@class="f8F1-price-text"]')
            type_voyage = extract_data(voyage, './/div[@class="aC3z-name"]')
            Compagnie = extract_data(voyage, './/div[@class="J0g6-operator-text"]')
        # Imprimer les résultats pour chaque voyage
            print("Horaires:", horaires)
            print("Trajet:", trajet)
            print("Type de trajet:", type_trajet)
            print("Temps de trajet:", temps_trajet)
            print("Prix:", prix)
            print("Type de voyage:", type_voyage)
            print("Compagnie de vol:", Compagnie)
            print("========================================")
        # Fermeture du webdriver
        driver.quit()

    # Passer à la semaine suivante
    start_date += timedelta(weeks=1)
    end_date += timedelta(weeks=1)

print('Fin du scraping')